In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import re
import nltk
from nltk.corpus import stopwords
import string
import json
from time import time
import pickle
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Embedding, LSTM
from keras.layers.merge import add
import collections

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
index_to_word = None
with open("index_to_word.pkl",'rb') as f:
  index_to_word = pickle.load(f)

In [0]:
word_to_index = None
with open("word_to_index.pkl",'rb') as f:
  word_to_index = pickle.load(f)

In [0]:
embedding_matrix = None
with open("embedding_matrix.pkl",'rb') as f:
  embedding_matrix = pickle.load(f)

In [0]:
encoding_train = None
with open("/content/gdrive/My Drive/Data_Science/Image_captioning_bot/30k/encoding_train_features.pkl",'rb') as f:
  encoding_train = pickle.load(f)

In [0]:
train_descriptions = None
with open("train_descriptions.pkl",'rb') as f:
  train_descriptions = pickle.load(f)

In [9]:
max_len = 0
for key in train_descriptions.keys():
    for cap in train_descriptions[key]:
        max_len = max(max_len,len(cap.split()))
print(max_len)

74


In [10]:
vocab_size = len(word_to_index)+1
print(vocab_size)

5121


In [0]:
def data_generator(train_descriptions,encoding_train,word_to_index,max_len,batch_size):
    X1,X2,Y = [],[],[]
    
    n=0
    while True:
        for key, desc_list in train_descriptions.items():
            n+=1
            
            photo = encoding_train[key]
            for desc in desc_list:
                seq = [word_to_index[word] for word in desc.split() if word in word_to_index]
                for i in range(1,len(seq)):
                    xi = seq[0:i]
                    yi = seq[i]
                    
                    xi = pad_sequences([xi],maxlen=max_len,value=0,padding='post')[0]
                    yi = to_categorical([yi],num_classes=vocab_size)[0]
                    
                    X1.append(photo)
                    X2.append(xi)
                    Y.append(yi)
                    
                if n==batch_size:
                    yield [[np.array(X1),np.array(X2)],np.array(Y)]
                    
                    X1,X2,Y = [],[],[]
                    n=0

In [0]:
input_img_features = Input(shape=(2048,))
inp_img1 = Dropout(0.1)(input_img_features)
inp_img2 = Dense(256,activation='relu')(inp_img1)

input_captions = Input(shape=(max_len,))
inp_cap1 = Embedding(input_dim=vocab_size,output_dim=50,mask_zero=True)(input_captions)
inp_cap2 = Dropout(0.1)(inp_cap1)
inp_cap3 = LSTM(256)(inp_cap2)

decoder1 = add([inp_img2,inp_cap3])
decoder2 = Dense(256,activation='relu')(decoder1)
outputs = Dense(vocab_size,activation='softmax')(decoder2)

model = Model(inputs=[input_img_features,input_captions],outputs=outputs)

In [13]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 74)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 74, 50)       256050      input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           input_1[0][0]                    
____________________________________________________________________________________________

In [0]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [0]:
epochs=40
batch_size = 64
steps = len(train_descriptions)//batch_size

In [0]:
model.compile(loss="categorical_crossentropy",optimizer="adam")

In [0]:
!mkdir model_weights

In [18]:
for i in range(epochs):
    generator = data_generator(train_descriptions,encoding_train,word_to_index,max_len,batch_size)
    model.fit_generator(generator,epochs=1,steps_per_epoch=steps,verbose=1)
    model.save('model_weights/model_'+str(i)+'.h5')

Epoch 1/1
496/496 [==============================] - 207s 418ms/step - loss: 4.7486
Epoch 1/1
496/496 [==============================] - 205s 414ms/step - loss: 3.9258
Epoch 1/1
496/496 [==============================] - 205s 413ms/step - loss: 3.6761
Epoch 1/1
496/496 [==============================] - 207s 417ms/step - loss: 3.5315
Epoch 1/1
496/496 [==============================] - 207s 417ms/step - loss: 3.4315
Epoch 1/1
496/496 [==============================] - 204s 412ms/step - loss: 3.3573
Epoch 1/1
496/496 [==============================] - 204s 410ms/step - loss: 3.2973
Epoch 1/1
496/496 [==============================] - 204s 412ms/step - loss: 3.2483
Epoch 1/1
496/496 [==============================] - 204s 411ms/step - loss: 3.2081
Epoch 1/1
496/496 [==============================] - 204s 412ms/step - loss: 3.1731
Epoch 1/1
496/496 [==============================] - 205s 413ms/step - loss: 3.1428
Epoch 1/1
496/496 [==============================] - 203s 410ms/step - loss: